# Replication of Main Figures / Tables

**Note**: See `README.md` in the main folder if you have not already, for instructions on how to generate the experiment results that this notebook uses.

This notebook will replicate tables in the main paper, and generate data for plotting figures, which is done via separate `R` scripts. 

After running this notebook end-to-end, run `plot_all.sh` which will run all of the relevant `R` scripts (`plot_figure_[2-5].R` and `plot_figure_s[2-3].R`) and populate `./figures/` with the relevant figures.

In [ ]:
# This flag can be set to use the same hyperparameters and thresholds 
# as in our published work.  Due to differences in features (described in README.md), there are still
# some minor differences, but this will more closely replicate our published results
USE_REP_HP = True

In [ ]:
import numpy as np
import pandas as pd
import os
import sys
from pathlib import Path

DATA_PATH = os.environ['DATA_PATH']
REPO_HOME = os.environ['REPO_PATH']
sys.path.insert(0, REPO_HOME)

EXP_PATH = os.environ['EXP_RESULT_PATH']
THRESH_TRAIN_PATH = os.environ['THRESHOLD_RESULT_PATH']
TRAIN_MODEL_PATH = os.environ['VAL_OUTCOME_MODEL_PATH']

if USE_REP_HP:
    PRED_PATH = os.environ['TEST_OUTCOME_MODEL_PATH_REP']
else:
    PRED_PATH = os.environ['TEST_OUTCOME_MODEL_PATH']

THRESH_TEST_PATH = f"{EXP_PATH}/thresholding/thresholding_eval_test/results"

In [ ]:
OUT_DIR = f"{REPO_HOME}/notebooks/fig_data"

In [ ]:
Path(OUT_DIR).mkdir(exist_ok=True)

In [ ]:
from analysis_utils import model_analysis_utils as ma_utils
from analysis_utils import policy_analysis_utils as policy_utils
from analysis_utils import best_case_baseline_utils as bca_utils

In [ ]:
# Get all data
test_features = pd.read_csv(f"{DATA_PATH}/test_uncomp_uti_features.csv")
train_features = pd.read_csv(f"{DATA_PATH}/train_uncomp_uti_features.csv")

test_labels = pd.read_csv(f"{DATA_PATH}/test_uncomp_resist_data.csv")
train_labels = pd.read_csv(f"{DATA_PATH}/train_uncomp_resist_data.csv")

In [ ]:
# Get train/test set predictions 
all_preds = pd.read_csv(f"{PRED_PATH}/test_predictions.csv")#.query('is_train == 0').drop(['is_train'], axis=1)
test_preds_actual = pd.merge(all_preds, test_labels, on='example_id')
train_preds_actual = pd.merge(all_preds, train_labels, on='example_id') 

In [ ]:
# Get all samples with previous exposure/resistance
test_prior_hist_eids = policy_utils.get_prior_exposure_examples(test_features)
test_prior_hist_preds_actual = test_preds_actual.query('example_id in @test_prior_hist_eids')

# Main Paper

First, we obtain the `policy_df`, which specifies the recommendations made by the algorithm.  This can either be loaded directly from the saved results of `thresholding_eval_test.sh`, or it can be manually re-constructed.  

We will do the latter here, in order to demonstrate both (a) the results obtained by running our code end-to-end, and (b) the results obtained by plugging in the thresholds chosen in the paper.

In [ ]:
import models.indirect.policy_learning_thresholding as plearn

abx_list=['NIT', 'SXT', 'CIP', 'LVX']

if not USE_REP_HP:
    val_outcomes_by_setting = pd.read_csv(f"{THRESH_TRAIN_PATH}/val_stats_by_setting.csv")

    constraint = 0.1

    best_setting = val_outcomes_by_setting[
        val_outcomes_by_setting['broad_prop'] <= constraint
    ].sort_values(by='iat_prop').iloc[0]

    curr_setting = dict(zip(abx_list, [{'vme': best_setting[abx]} for abx in abx_list]))

    # Note that we choose the VME (i.e., false susceptibility rate) using the validation splits
    # and then the "optimal threshold" corresponding to this VME is re-computed across the entire
    # training set
    thresholds = plearn.get_thresholds_dict(train_preds_actual, curr_setting, abx_list=abx_list)

    print(curr_setting)
    print(thresholds)
else:
    # Due to the nuances of train/validate splits being different in replication,
    # the above thresholds are different than those used in the published results.
    # The published thresholds are given below, and can be used to more closely replicate our published figures.

    # However, note that the results will still not be exactly the same, in part due to slight differences in the features, 
    # (as noted above) and therefore the trained models
    thresholds = {
        'NIT': 0.129,
        'SXT': 0.180,
        'CIP': 0.258,
        'LVX': 0.239,
    }

In [ ]:
policy_df = plearn.get_policy_for_preds(test_preds_actual, thresholds,
                                             abx_list=abx_list)

In [ ]:
if not USE_REP_HP:
    # We can verify that this policy_df matches the one saved on disk (assuming that we are NOT using the published thresholds)
    policy_df_saved = pd.read_csv(f"{THRESH_TEST_PATH}/test_policy_df.csv")
    assert np.all(np.equal(policy_df['rec_final'], policy_df_saved['rec_final']))

## Figures

Figure 1 (Schematic of analytic protocol) is not replicated here, as it is a conceptual figure

### Figure 2: Threshold sensitivity analysis

Note: This figure is only generated if you are using the end-to-end analysis

In [ ]:
if not USE_REP_HP:
    val_outcomes_by_setting.to_csv(f"{OUT_DIR}/figure_2_threshold_sensitivity.csv")

    # These values are plugged into the corresponding R script
    print(f"IAT: {best_setting['iat_prop']}\nSpectrum: {best_setting['broad_prop']}")

### Figure 3: False susceptibility and non-susceptibility rates

In [ ]:
fpr_fnr_df = ma_utils.create_fpr_fnr_data(test_preds_actual)
fpr_fnr_df.to_csv(f"{OUT_DIR}/figure_3_fpr_fnr.csv")

In [ ]:
thresholds_plot = {}
for name in [('Nitrofurantoin', 'NIT'), 
             ('TMP-SMX', 'SXT'),
             ('Ciprofloxacin', 'CIP'),
             ('Levofloxacin', 'LVX')]:
    thresholds_plot[name[0]] = thresholds[name[1]]

In [ ]:
thresh_df = pd.DataFrame.from_dict(thresholds_plot, orient='index').reset_index()
thresh_df = thresh_df.rename(columns={'index': 'drug', 0: 'value'})
thresh_df.to_csv(f"{OUT_DIR}/figure_3_chosen_thresh.csv")

### Figure 4: Post hoc analysis of clinician vs algorithm therapy decisions and appropriateness in patients with uncomplicated UTI presenting between 2014 and 2016. 

In [ ]:
breakdown_df = policy_utils.get_doc_alg_breakdown(policy_df)

In [ ]:
template_df = pd.DataFrame(np.zeros((16, 5)))
template_df = template_df.rename(columns = {
    0: "Decision-maker",
    1: "Drug",
    2: "Result",
    3: "Comparator",
    4: "Value"
})

i = 0
for compare in ['MD_narrow_IAT', 'MD_narrow_noIAT', 'MD_broad_IAT', 'MD_broad_noIAT']:
    for line in ['First line', 'Second line']:
        for res in ['Inappropriate', 'Appropriate']:
            template_df.iloc[i] = ('Model', line, res, compare, -1)
            i+= 1

In [ ]:
error_analysis_df = policy_utils.format_breakdown_df(template_df, breakdown_df)

In [ ]:
error_analysis_df.to_csv(f"{OUT_DIR}/figure_4_error_analysis.csv")

### Figure 5: Feature importance characterization

NOTE: This takes a few minutes to run, because it performs the relevant analysis.

In [ ]:
import json

if USE_REP_HP:
    with open(f"{REPO_HOME}/models/replication_hyperparameters/best_models.json") as f:
        best_model_class = json.load(f)

    with open(f"{REPO_HOME}/models/replication_hyperparameters/hyperparameters.json") as f:
        best_params = json.load(f)
else:
    with open(f"{TRAIN_MODEL_PATH}/best_models.json") as f:
        best_model_class = json.load(f)

    with open(f"{TRAIN_MODEL_PATH}/hyperparameters.json") as f:
        best_params = json.load(f)

figure_5_df = policy_utils.run_feature_importance_analysis(
    train_features, train_labels,
    test_features, test_labels,
    model_class_dict = best_model_class, 
    best_params_dict = best_params)

figure_5_df.to_csv(f"{OUT_DIR}/figure_5.csv")

## Tables

Table 1 is not replicated here, as the statistics are computed on a per-patient basis, as opposed to a per-sample basis, and patient identifiers are not included in this dataset release

### Table 2: AUROCs for prediction of antibiotic non-susceptibility in patients presenting with uncomplicated UTI between 2014 and 2016

In [ ]:
from models.indirect.train_outcome_models import evaluate_test_cohort

import json

if USE_REP_HP:
    with open(f"{REPO_HOME}/models/replication_hyperparameters/best_models.json") as f:
        best_model_class = json.load(f)

    with open(f"{REPO_HOME}/models/replication_hyperparameters/hyperparameters.json") as f:
        best_params = json.load(f)
else:
    with open(f"{TRAIN_MODEL_PATH}/best_models.json") as f:
        best_model_class = json.load(f)

    with open(f"{TRAIN_MODEL_PATH}/hyperparameters.json") as f:
        best_params = json.load(f)
        
abx_name_map = {
    'NIT': 'Nitrofurantoin',
    'SXT': 'TMP-SMX',
    'CIP': 'Ciprofloxacin',
    'LVX': 'Levofloxacin',
}

test_prior_hist_eids = policy_utils.get_prior_exposure_examples(test_features)
train_prior_hist_eids = policy_utils.get_prior_exposure_examples(train_features)

prior_hist_eids = np.concatenate([train_prior_hist_eids, test_prior_hist_eids])

In [ ]:
table_2 = []
auc_dict = {}

auc_dict['full'], _ , _ = evaluate_test_cohort(train_features, train_labels,
                                                      test_features, test_labels, 
                                                      best_params, best_model_class,
                                                      subcohort_eids=None)

auc_dict['prior'], _ , _ = evaluate_test_cohort(train_features, train_labels,
                                                      test_features.query('example_id in @test_prior_hist_eids'), 
                                                      test_labels.query('example_id in @test_prior_hist_eids'), 
                                                      best_params, best_model_class,
                                                      subcohort_eids=None)

In [ ]:
for cohort, auc_result in auc_dict.items():
    for drug_code, drug_name in abx_name_map.items():
        mean_auc, stdev_auc, ci_auc = auc_result[drug_code]

        table_2.append([cohort, drug_name,
                      mean_auc.round(2), ci_auc[0].round(2), ci_auc[1].round(2)])

In [ ]:
table_2

### Table 3: Comparison of primary outcomes for algorithm, clinicians and best-case guideline-based policy in patients presenting with uncomplicated UTI between 2014 and 2016.

In [ ]:
policy_df = policy_df.rename(
        columns={'rec': 'alg_raw', 'rec_final': 'alg', 'prescription': 'doc'}
    )

prev_resist_NIT = list(test_features[test_features['micro - prev resistance NIT 90'] == 1].example_id.values)
prev_exposure_NIT = list(test_features[test_features['medication 90 - nitrofurantoin'] == 1].example_id.values)
avoid_NIT_eids = set(prev_resist_NIT).union(set(prev_exposure_NIT))

policy_df['idsa'] = policy_df.apply(lambda x: 'CIP' if x['example_id'] in set(avoid_NIT_eids) else 'NIT',
                                   axis=1)

In [ ]:
table_3a = policy_utils.compile_all_stats(policy_df)

# Get the decisions for the modified guideline
policy_df_decision = policy_df[policy_df['alg_raw'] != 'defer']
table_3b_all = bca_utils.get_best_case_stats(policy_df, p=.18)
table_3b_decision = bca_utils.get_best_case_stats(policy_df_decision, p=.18)

In [ ]:
table_dict = {
    'broad': {
        'decision': {},
        'all': {}
    },
    'iat': {
        'decision': {},
        'all': {}
    },
}

best_guideline = {
    'decision': {
        'iat': table_3b_decision[1]['all'],
        'broad': table_3b_decision[0]['all']
    }, 
    'all': {
        'iat': table_3b_all[1]['all'],
        'broad': table_3b_all[0]['all']        

    }
}

for policy in ['alg', 'doc', 'best_case_guideline']:
    for cohort in ['decision', 'all']:
        if policy == 'best_case_guideline':
            table_dict['iat'][cohort][policy] = best_guideline[cohort]['iat']
            table_dict['broad'][cohort][policy] = best_guideline[cohort]['broad']            
        else:
            table_dict['iat'][cohort][policy] = table_3a.query(
                "policy == @policy & subcohort == @cohort & antibiotic == 'all'")[
                ['mean_iat', 'ci_iat']].round(3).values[0].tolist()
            table_dict['broad'][cohort][policy] = table_3a.query(
                "policy == @policy & subcohort == @cohort & antibiotic == 'second'")[
                ['mean_abx', 'ci_abx']].round(3).values[0].tolist()

In [ ]:
decision_cohort_size = table_3a.query('policy == "alg" & subcohort == "decision" & antibiotic == "all"')['n'].values[0]
full_cohort_size = table_3a.query('policy == "alg" & subcohort == "all" & antibiotic == "all"')['n'].values[0]

In [ ]:
print("Table 3")
for stat in ['broad', 'iat']:
    print(f"\t {stat}")
    for cohort in ['decision', 'all']:
        print(f"\t\t {cohort}: {decision_cohort_size if cohort == 'decision' else full_cohort_size}")
        for policy in ['alg', 'doc', 'best_case_guideline']:
            print(f"\t\t\t {policy}: {table_dict[stat][cohort][policy]}")

# (Selected) Supplementary Materials

## Supplementary Figures

Figure S1 is not replicated here, as we do not provide time information in the dataset release

### Figure S2: Test Set ROCs

In [ ]:
figure_s2_df = ma_utils.create_roc_curve_data(test_preds_actual, test_prior_hist_preds_actual)
figure_s2_df.to_csv(f"{OUT_DIR}/figure_s2.csv")

### Figure S3: Calibration plots

In [ ]:
figure_s3_df = ma_utils.create_calibration_data_df(test_preds_actual)
figure_s3_df.to_csv(f"{OUT_DIR}/figure_s3.csv")

### Figures S4-S5

Figure S4 (Cohort selection) is not replicated here, as it requires data (e.g., patient identifiers) that is not present in the data release. 

Figure S5 (Detailed methods schematic) is not replicated here, as it is just a conceptual schematic

## Supplementary Tables

### Table S1: Model Hyperparameters

In [ ]:
if USE_REP_HP:
    with open(f"{REPO_HOME}/models/replication_hyperparameters/hyperparameters.json") as f:
        best_params = json.load(f)
else:
    with open(f"{TRAIN_MODEL_PATH}/hyperparameters.json") as f:
        best_params = json.load(f)
        
print(best_params)

### Table S2: Detailed breakdown of primary outcomes

In [ ]:
# This constructs the conservative IDSA guidelines
prev_resist_NIT = list(test_features[test_features['micro - prev resistance NIT 90'] == 1].example_id.values)
prev_exposure_NIT = list(test_features[test_features['medication 90 - nitrofurantoin'] == 1].example_id.values)
avoid_NIT_eids = set(prev_resist_NIT).union(set(prev_exposure_NIT))

policy_df['idsa'] = policy_df.apply(lambda x: 'CIP' if x['example_id'] in set(avoid_NIT_eids) else 'NIT',
                                   axis=1)

# This will give all relevant stats for a variety of antibiotic combinations, for
# * Clinicians
# * Algorithm
# * Conservative IDSA guidelines
table_s2 = policy_utils.compile_all_stats(policy_df)

# To get decisions for the modified IDSA guideline (the "best-case guidelines"), you will need to use this function
# bca_utils.get_best_case_stats(policy_df, p=.18)

# To filter the above to a particular cohort, you can pass in a different cohort, e.g.,
# policy_df_decision = policy_df[policy_df['alg_raw'] != 'defer']

### Table S3: Features predicting use of fluoroquinolones

This table is not replicated here

### Table S4: Top 10 features for prediction of non-susceptibility

In [ ]:
top_coef_dict = {}
for abx in ['NIT', 'SXT', 'CIP', 'LVX']:
    top_coef_dict[abx] = pd.read_csv(f"{PRED_PATH}/pos_coeffs_{abx}.csv").round(2).values
    print(top_coef_dict[abx])